<a href="https://colab.research.google.com/github/mcstllns/DeepLearning_2025/blob/main/PRACTICA_07_Trabajo_con_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="darkorange" size="10"><b>07. Trabajo con APIs</b></font>

Miguel A. Castellanos



**Contenidos:**

<font color="darkorange">

1. Clasificación de textos
  1. Clasificación Sentiment-Analysis
  1. Clasificación zero-shot
1. Gestión de los modelos
1. Generación de Textos
1. Respuestas a preguntas
1. Resumir Textos
1. Traducción
1. ChatBot
1. Embeddings
1. Algunos datasets interesantes
1. Cómo entrenar una red para una tarea específica (Fine-Tuning)

</font>

Una API (Application Programming Interface o Interfaz de Programación de Aplicaciones) es un conjunto de reglas y definiciones que permiten que diferentes aplicaciones o sistemas se comuniquen entre sí.

Una API actúa como un intermediario que facilita la interacción entre diferentes programas o servicios. Funciona enviando solicitudes y recibiendo respuestas en formatos como JSON o XML.

La API Transformers de Hugging Face es una biblioteca de código abierto en Python que permite el uso de modelos de procesamiento de lenguaje natural (NLP) y otros tipos de modelos de inteligencia artificial basados en redes neuronales profundas, como los Transformers.

Facilita la implementación de modelos avanzados de IA, como GPT, BERT, T5, RoBERTa y muchos otros, para tareas como:

- Análisis de sentimientos: Determina la polaridad de un texto, identificando si es positivo, negativo o neutral.​
- Clasificación de texto: Asigna categorías predefinidas a fragmentos de texto, como etiquetar correos electrónicos como "spam" o "no spam".​
- Reconocimiento de entidades nombradas (NER): Identifica y clasifica entidades como nombres de personas, organizaciones o lugares dentro de un texto.​
- Respuesta a preguntas: Proporciona respuestas precisas basadas en un contexto textual dado.​
- Rellenado de máscaras: Completa palabras faltantes en una oración con las opciones más probables.​
- Resumir textos: Genera resúmenes concisos de documentos o artículos extensos.​
- Traducción automática: Traduce texto de un idioma a otro.​
- Generación de texto: Produce texto coherente y contextual a partir de una entrada inicial.

Y otras muchas más tareas.


ChatGPT no es una AGI pero lo simula porque tiene implementadas todas esas tareas y el usuario la percibe como unitaria.

La API de HuggingFace incorpora la mayoría de los modelo open que existen como:

- BERT (Bidirectional Encoder Representations from Transformers): Modelo preentrenado que captura el contexto de una palabra considerando tanto su izquierda como su derecha en una oración.​

- GPT (Generative Pre-trained Transformer): Modelo de lenguaje generativo que puede producir texto coherente y relevante en función de una entrada dada.​

- RoBERTa (Robustly Optimized BERT Approach): Una variante de BERT optimizada con más datos y técnicas de entrenamiento mejoradas.​

- T5 (Text-To-Text Transfer Transformer): Modelo que convierte todas las tareas de procesamiento de texto en un formato de entrada-salida de texto a texto.​

- DistilBERT: Versión reducida y más rápida de BERT que mantiene el 97% de su rendimiento con menos parámetros.​

En total tiene unos 400.000 modelos, se pueden consultar en [https://huggingface.co/models](https://huggingface.co/models)


Muchos de esos modelos han sido adaptados y entrenados al español.

Consejo: no busques por castellano como palabra clave, busca por español o spanish, fuera de España a nadie le importa nuestro problemas idiomáticos.

Vamos a ver las siguientes tareas, en cada una de ellas utilizaremos que, sin ser muy demandante (recuerda que colab es gratis y está limitado) permita realizar decentemente la tarea


En algunas tareas nos vamos a basar en el uso de pipelines, en otras, las más complejas, cargaremos y ejecutaremos el código de forma más básica

En la biblioteca Transformers de Hugging Face, una pipeline es una herramienta que simplifica el uso de modelos preentrenados para realizar tareas específicas de procesamiento de lenguaje natural, visión por computadora y audio. Esta abstracción permite aplicar modelos complejos sin necesidad de profundizar en los detalles técnicos de preprocesamiento, tokenización o normalización de datos.


Muchos de estos modelos, aunque son mínimos comparados con modelos "avanzados", solo se pueden usar a través de la GPU, por lo tanto arranca la T4 para poder realizar esta práctica.

Si realizas todas las tareas de seguido, lo más probable es que al final Google te eche del entorno de ejecución porque has alcanzado la cuota gratis.

In [ ]:
# Importante, vamos a lanzarlo todo sobre la GPU o será imposible ejecutar los modelos

import torch
from transformers import pipeline

# Verificar si la GPU está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

In [ ]:
# Opcional
# puede ayudar a acelerar los calculos en HF pero también puede tardar bastante en instalarse
# !pip install accelerate;

In [ ]:
from transformers import pipeline

# Cargar el modelo RoBERTa y asegurarse de que se ejecute en la GPU
device = 0 if torch.cuda.is_available() else -1  # 0 para usar GPU, -1 para usar CPU
print(device)


<font color="darkorange" size="6"><b>01. Clasificación de textos</b></font>

<font color="darkorange" size="5"><b>Clasificación Sentiment-Analysis</b></font>


Vamos a hacer un análisis de opiniones (Ojo, en inglés sentiment no es solo sentimientos). Se usa un modelo ya entrenado para ello en español.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# Cargar tokenizador y modelo por separado
model_name = "pysentimiento/robertuito-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Crear pipeline usando el modelo y tokenizador ya cargados
clasificador = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

In [ ]:
# Textos para analizar
textos = [
    "Estoy encantado con la compra, todo fue perfecto.",
    "No me gustó el producto, esperaba algo mejor.",
    "La atención fue aceptable, pero nada extraordinario."
]

# Realizar análisis de sentimientos
resultados = clasificador(textos)

# Mostrar resultados
for texto, resultado in zip(textos, resultados):
    print(f"Texto: {texto}\nSentimiento: {resultado['label']} ({resultado['score']:.3f})\n")


<font color="darkorange" size="5"><b>Clasificación zero-shot</b></font>


Zero shot se refiere a una tarea en la que el modelo no ha tenido entrenamiento previo con ejemplos específicos para esa tarea. Esto es, eneste caso una clasificacion para la que no ha sido entrenada la red.

Una utilidad es la de clasificación de textos en función de eiquetas predefinidas.

Obviamente la "capacidad" y el tamaño del modelo (los Billones) es determinante. Cuanto mayor sea el modelo más probable es que la realice bien sin entrenamiento previo.



In [ ]:
# Cargar la pipeline de clasificación zero-shot

model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

clasificador = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=device)

In [ ]:
# Texto a clasificar
text = "I love programming, it's my passion!"

# Etiquetas candidatas
candidate_labels = ["technology", "sports", "politics", "entertainment", "love", "hate", "positive", "negative"]

# Clasificación
result = clasificador(text, candidate_labels)

# Mostrar los resultados
print(result)

In [ ]:
# En castellano funciona razonablemente bien

# Texto a clasificar
text = "Me gustan los helados de fresa"

# Etiquetas candidatas
# candidate_labels = ["technology", "sports", "politics", "entertainment", "love", "hate", "positive", "negative", "summer", "winter"]
candidate_labels = ["tecnologia", "deporte", "politica", "entretenimiento", "amor", "odio", "positivo", "negativo", "verano", "invierno"]

# Clasificación
result = clasificador(text, candidate_labels)

# Mostrar los resultados
print(result)

<font color="darkorange" size="6"><b>02. Gestión de los modelos</b></font>

Si vamos descargando en disco y cargando en memoria modelos y modelos llegará un momento en el que la ejecución sea muy lenta o no podemos ejecutar nada, por eso es importante llevar una cierta "higiene" en el entorno de ejecución.

A la derecha, en recursos, podemos ver la evolución de la carga de memoria (RAM y VRAM) y del disco duro

Vamos a ver cómo eliminar de la memoria VRAM un modelo y luego eliminarlo del disco

In [ ]:
# Ver la memoria RAM
!free -h

In [ ]:
# Ver lo que nos dice la nvidia
!nvidia-smi

In [ ]:
# primero, liberar el modelo de la memoria de la GPU (VRAM)

del model, tokenizer, clasificador
# del model

# Libera la memoria de la GPU
torch.cuda.empty_cache()

# Llama al colector de basura y elimina todo lo que vea que está suelto por ahí
import gc
gc.collect()


In [ ]:
# A veces despues de hacerlo se sigue quedando igual la memoria
# esto es porque pytorch reserva memoria para futuras ejecuciones

# Memoria asignada actualmente por tensores
print(f"Memoria asignada (GB): {torch.cuda.memory_allocated() / (1024 ** 3):.3f}")

# Memoria total reservada por PyTorch
print(f"Memoria reservada (GB): {torch.cuda.memory_reserved() / (1024 ** 3):.3f}")


Y hay veces que para eliminar del todo lo que hay en el VRAM hay que ir a Entorno de ejecución -> Reiniciar la sesión



In [ ]:
# Se instala el CLI de HF
!pip install huggingface_hub[cli]

In [ ]:
# vemos los modelos almacenados en cache
!huggingface-cli scan-cache

In [ ]:
from huggingface_hub import scan_cache_dir

# Función de búsqueda del hash del modelo
def busca_hash(modelo, cache_info):
    for repo_info in cache_info.repos:
        if repo_info.repo_id == modelo:
            print(f"Modelo: {repo_info.repo_id}")
            for revision in repo_info.revisions:
                print(f"  Hash del commit: {revision.commit_hash}")
                print(f"  Referencias: {revision.refs}")
            return revision.commit_hash
    return None



cache_info = scan_cache_dir()
hash = busca_hash("facebook/bart-large-mnli", cache_info)

if hash:
    delete_strategy = cache_info.delete_revisions(hash)
    # Mostrar el espacio que se liberará
    print(f"Se liberarán {delete_strategy.expected_freed_size_str}.")
    # Ejecutar la eliminación
    delete_strategy.execute()
else:
    print("No se encontró el modelo especificado en la caché.")

In [ ]:
del hash, delete_strategy

In [ ]:
# Si vuelves a ejecutar el scan-cache el modelo ha desaparecido
!huggingface-cli scan-cache

<font color="darkorange" size="6"><b>03. Generación de Textos</b></font>

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "DeepESP/gpt2-spanish-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

In [ ]:
entrada = "En un futuro distópico, la inteligencia artificial ha"
inputs = tokenizer(entrada, return_tensors="pt").to(device) # devuelve tensores de pytorch

In [ ]:
# parámetros del modelo

outputs = model.generate(
    inputs.input_ids,
    max_length=200,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    do_sample=True,
    early_stopping=True
)

Los parametros son:


| Parámetro                | Descripción                                                                    | Efecto                                |
|--------------------------- |--------------------------------------------------------------------------------|---------------------------------------|
| `max_length=200`           | Longitud máxima del texto generado (incluye texto inicial).  | Genera texto con máximo 200 tokens.   |
| `num_return_sequences=1`   | Número de secuencias generadas.                              | Genera solo una secuencia.            |
| `no_repeat_ngram_size=2`   | Evita repetir frases (n-gramas) en el texto generado.        | Evita repeticiones de bigramas.       |
| `temperature=0.7`          | Controla la aleatoriedad (0 determinista, mayor a 1 muy creativo). | Equilibrio entre coherencia y diversidad. |
| `top_p=0.9`                | Usa palabras cuya probabilidad acumulada alcanza este valor. | Usa palabras más probables hasta el 90%.|
| `top_k=50`                 | Considera las 50 palabras más probables en cada paso.        | Mantiene calidad, evita palabras raras.|
| `do_sample=True`           | Activa sampling probabilístico al seleccionar palabras.      | Genera textos diversos y variados.    |
| `early_stopping=True`      | Finaliza la generación al alcanzar condiciones de parada.    | Detiene la generación automáticamente.|


In [ ]:
texto_generado = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(texto_generado)

In [ ]:
entrada = "los helados de fresa"
inputs = tokenizer(entrada, return_tensors="pt").to(device)
outputs = model.generate(
      inputs.input_ids,
      max_length=200,
      num_return_sequences=1,
      no_repeat_ngram_size=2,
      temperature=0.3,
      top_p=0.9,
      top_k=50,
      do_sample=True,
      early_stopping=True
  )
texto_generado = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(texto_generado)

<font color="darkorange" size="6"><b>04. Respuestas a preguntas</b></font>

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline


# Cargar el mejor modelo disponible en español para QA
modelo_nombre = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

# Cargar modelo y tokenizador
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
modelo = AutoModelForQuestionAnswering.from_pretrained(modelo_nombre).to(device)

# Crear el pipeline de QA con el modelo cargado
qa_pipeline = pipeline("question-answering", model=modelo, tokenizer=tokenizer, device=device)

# Contexto y pregunta en español
contexto = """
Lionel Messi es un futbolista argentino nacido en Rosario.
Ha ganado siete Balones de Oro y es ampliamente considerado uno de los mejores jugadores de fútbol de todos los tiempos.
Jugó muchos años en el FC Barcelona antes de transferirse al Paris Saint-Germain en 2021.
"""

# ---------------------------------------------
pregunta = "¿Dónde juega Lionel Messi actualmente?"

# Realizar la predicción
resultado = qa_pipeline(question=pregunta, context=contexto)

# Mostrar respuesta
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {resultado['answer']} (Confianza: {resultado['score']:.3f})")

# ---------------------------------------------
pregunta = "¿Cuántos balones de oro tiene?"

# Realizar la predicción
resultado = qa_pipeline(question=pregunta, context=contexto)

# Mostrar respuesta
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {resultado['answer']} (Confianza: {resultado['score']:.3f})")

# ---------------------------------------------
pregunta = "¿Jugó en el Real Madrid?"

# Realizar la predicción
resultado = qa_pipeline(question=pregunta, context=contexto)

# Mostrar respuesta
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {resultado['answer']} (Confianza: {resultado['score']:.3f})")

<font color="darkorange" size="6"><b>05. Resumir Textos</b></font>


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, logging
import torch

# Modelo especializado para resúmenes en español
modelo_nombre = "mrm8488/bert2bert_shared-spanish-finetuned-summarization"

# Silenciar mensajes no críticos
logging.set_verbosity_error()

# Cargar modelo y tokenizador por separado
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
modelo = AutoModelForSeq2SeqLM.from_pretrained(modelo_nombre).to(device)

# Crear pipeline de resumen (corregido aquí)
resumidor = pipeline("summarization", model=modelo, tokenizer=tokenizer, device=device)

# Texto largo que deseas resumir
texto_largo = """
OpenAI es una empresa dedicada a la investigación en inteligencia artificial (IA) con sede en San Francisco, California.
Fue fundada en 2015 por Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman y Wojciech Zaremba,
con el objetivo de desarrollar tecnologías avanzadas de IA de manera segura y beneficiosa para toda la humanidad.
OpenAI ha desarrollado tecnologías reconocidas mundialmente, como GPT-3 y ChatGPT, herramientas ampliamente usadas en diversos campos.
"""

# Realizar resumen
resumen = resumidor(texto_largo, max_length=50, min_length=20, do_sample=False)

# Mostrar resultado
print(f"Resumen: {resumen[0]['summary_text']}")


<font color="darkorange" size="6"><b>06. Traducción</b></font>

In [ ]:
# opcinal, acelera calculos
# !pip install sentencepiece

In [ ]:
from transformers import pipeline

translator_es_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

texto_espanol = "Estoy aprendiendo a traducir textos usando inteligencia artificial."
traduccion_ingles = translator_es_en(texto_espanol)[0]['translation_text']

print("Español:", texto_espanol)
print("Inglés:", traduccion_ingles)

<font color="darkorange" size="6"><b>07. ChatBot</b></font>

In [ ]:
# Instalar dependencias necesarias
# Instalar las cosas y descargar el modelo puede llevar bastante tiempo
# Pero la aplicación queda muy chula con gradio

!pip install -q transformers accelerate sentencepiece gradio

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr
import warnings
warnings.filterwarnings("ignore")
import gc

class ChatbotEspanol:
    def __init__(self):
        # Verificar si hay GPU disponible en Colab
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Usando dispositivo: {self.device}")

        # Cargar el modelo Gemma 2B adaptado para español
        # Este es uno de los mejores modelos livianos disponibles para español que funcionará bien en Colab
        self.model_name = "LenguajeNaturalAI/leniachat-gemma-2b-v0"

        # Inicializar el tokenizer
        print("Cargando tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Configuración para optimizar el uso de memoria en Colab
        print("Cargando modelo...")
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            low_cpu_mem_usage=True,
            device_map="auto"
        )

        # Inicializar el historial de conversación
        self.history = []
        print("¡Chatbot listo!")

    def generate_response(self, user_input, history):
        # Convertir el historial de Gradio al formato que necesitamos
        self.history = []
        for human, assistant in history:
            self.history.append({"role": "user", "content": human})
            if assistant:  # Puede ser None en la primera interacción
                self.history.append({"role": "assistant", "content": assistant})

        # Añadir el input actual del usuario
        self.history.append({"role": "user", "content": user_input})

        # Preparar la conversación formateada para el modelo
        conversation = []
        for message in self.history:
            conversation.append({"role": message["role"], "content": message["content"]})

        # Generar tokens con la plantilla de chat adecuada
        messages = self.tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=True
        )

        # Tokenizar la conversación
        inputs = self.tokenizer(messages, return_tensors="pt").to(self.device)

        # Generar respuesta
        outputs = self.model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )

        # Obtener la respuesta generada
        response = self.tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

        # Limpiar memoria CUDA para evitar fugas de memoria en Colab
        if self.device == "cuda":
            del inputs, outputs
            gc.collect()
            torch.cuda.empty_cache()

        return response

# Inicializar el chatbot
chatbot = ChatbotEspanol()

In [ ]:
# Crear la interfaz con Gradio
with gr.Blocks(theme=gr.themes.Soft(primary_hue="green")) as demo:
    gr.Markdown("# 🤖 Chatbot en Español con Hugging Face")
    gr.Markdown("Este chatbot utiliza el modelo mlabonne/gemma-2b-it-spanish para generar respuestas en español.")

    chatbot_interface = gr.ChatInterface(
        chatbot.generate_response,
        title="Conversación",
        examples=[
            "¿Cómo estás hoy?",
            "Explícame el calentamiento global como si tuviera 10 años",
            "¿Qué libros recomiendas para aprender sobre inteligencia artificial?",
            "Escribe un poema corto sobre la primavera",
            "Dame tres ideas para mejorar mi productividad"
        ]
        # retry_btn="Reintentar",
        # undo_btn="Deshacer",
        # clear_btn="Limpiar"
    )

# Lanzar la interfaz
demo.launch(debug=False, share=True)

In [ ]:
demo.close()

<font color="darkorange" size="6"><b>08. Embeddings</b></font>


​Los embeddings son representaciones matemáticas que transforman datos complejos y de alta dimensión, como palabras, imágenes o sonidos, en vectores de números reales en un espacio de menor dimensión. Esta transformación facilita el procesamiento y análisis de la información, preservando relaciones y patrones significativos entre los datos.

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# Verificar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}")

# Función para aplicar Mean Pooling sobre las representaciones del modelo
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # Última capa oculta
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Frases en español a convertir en embeddings
sentences = ["¿Cómo está el clima hoy?", "¿Qué tiempo hace hoy?", "¿Cuál es la capital de España?"]

# Cargar el tokenizador y el modelo en la GPU
model_name = "jinaai/jina-embeddings-v2-base-es"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# Tokenizar las frases y moverlas a la GPU
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)

# Obtener embeddings con el modelo (sin calcular gradientes)
with torch.no_grad():
    model_output = model(**encoded_input)

# Aplicar Mean Pooling para obtener una única representación por frase
embeddings = mean_pooling(model_output, encoded_input["attention_mask"])

# Normalizar los embeddings (para cálculos de similitud)
embeddings = F.normalize(embeddings, p=2, dim=1)

# Mostrar los embeddings generados
print("Embeddings generados:")
print(embeddings)

print(embeddings.shape)

In [ ]:
print(embeddings.shape)

In [ ]:
similarity_matrix = torch.matmul(embeddings, embeddings.T)

# Convertir a numpy para visualizar mejor
import numpy as np
import pandas as pd
# import ace_tools as tools

similarity_matrix_np = similarity_matrix.cpu().numpy()

# Crear un DataFrame para mostrar la matriz de similitud
df_similarity = pd.DataFrame(similarity_matrix_np, index=sentences, columns=sentences)

# Mostrar la matriz de similitud
# tools.display_dataframe_to_user(name="Matriz de Similitud", dataframe=df_similarity)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.heatmap(df_similarity, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Similitud del Coseno")
plt.show()


<font color="darkorange" size="6"><b>09. Algunos datasets interesantes</b></font>


Aquí tienes un listado de algunos de los mejores datasets en español disponibles gratuitamente para trabajar con modelos de lenguaje (LLMs):

1. MLSUM - Corpus de resúmenes de noticias en español con más de 250,000 artículos de El País.
1. Spanish Billion Words Corpus - Colección masiva de textos en español de diversas fuentes web.
1. MC4-es - Subset en español del Multilingual C4 de Google, con textos extraídos de Common Crawl.
1. OPUS - Colección de textos paralelos traducidos en múltiples idiomas, incluyendo español.
1. esWiki-abstracts - Abstracts de la Wikipedia en español.
1. SQAC (Spanish Question Answering Corpus) - Dataset de preguntas y respuestas en español.
1. XNLI-es - Versión en español del dataset de inferencia de lenguaje natural.
1. PAWS-X-es - Corpus para evaluación de paráfrasis en español.
1. HC3-Spanish - Dataset de conversaciones humano-chatbot en español para detectar texto generado.
1. CardioSentiBr - Opiniones de pacientes sobre hospitales en español.
1. TASS - Corpus de tweets en español con análisis de sentimiento.
1. MeliSA - Dataset de opiniones de productos de Mercado Libre con análisis de sentimiento.
1. CAPITEL - Corpus anotado de la Real Academia Española.
1. Spanish SQuAD - Versión traducida del dataset SQuAD para respuesta a preguntas.
1. MarIA datasets - Colección de datasets para el entrenamiento de modelos en español.

1. Conjunto de Datos de Reseñas de Amazon
Este dataset recopila reseñas de productos de Amazon en diversos idiomas, incluyendo el español. Es ampliamente utilizado para tareas de clasificación de texto y análisis de sentimientos.

1. Conjunto de Datos de Twitter para Análisis de Sentimientos
Existen múltiples datasets que recopilan tuits en español, etiquetados según el sentimiento expresado. Estos conjuntos de datos son valiosos para entrenar modelos en tareas de análisis de sentimientos y detección de emociones.

1. Corpus de Mensajes de Foros y Redes Sociales
Algunas universidades y centros de investigación han compilado conjuntos de datos que incluyen mensajes de foros y redes sociales en español, útiles para analizar interacciones y comportamientos en línea.

1. Corpus de Noticias en Español
Recopilaciones de artículos de noticias en español, provenientes de diversas fuentes, que pueden ser utilizadas para tareas de resumen automático, clasificación de noticias y detección de temas.

1. Conjunto de Datos de Comentarios en Redes Sociales
Algunas plataformas académicas ofrecen datasets que contienen comentarios y publicaciones de redes sociales en español, útiles para el análisis de opiniones y estudios sociológicos.

1. Corpus de Textos Legales en Español
Compilaciones de documentos legales y jurídicos en español, que pueden ser utilizadas para entrenar modelos especializados en el ámbito legal.

1. Corpus de Conversaciones en Español
Datasets que recopilan conversaciones y chats en español, útiles para entrenar modelos de diálogo y chatbots.


La mayoría de estos datasets están disponibles a través de Hugging Face Datasets, el Linguistic Data Consortium o los repositorios de sus creadores. Para casos específicos como redes sociales, algunos investigadores han publicado datasets anonimizados que cumplen con los términos de servicio de las plataformas.

Además existe el paquete datasets de HuggingFace que incluye unas 300.000 bases, casi todas en inglés, aunque no solamente

[https://huggingface.co/datasets?p=99&sort=trending](https://huggingface.co/datasets?p=99&sort=trending)

In [ ]:
!pip install datasets

In [38]:
# dataset de reseñas de amazon (en inglés)
from datasets import load_dataset

# Cargar el conjunto de datos de reseñas de Amazon
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)

# Mostrar la primera reseña
print(dataset["full"][0])

{'rating': 5.0, 'title': 'Such a lovely scent but not overpowering.', 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!", 'images': [], 'asin': 'B00YQ6X8EO', 'parent_asin': 'B00YQ6X8EO', 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ', 'timestamp': 1588687728923, 'helpful_vote': 0, 'verified_purchase': True}


In [ ]:
# dataset de amazon en español
dataset = load_dataset("SetFit/amazon_reviews_multi_es")

In [ ]:
# Mostrar las primeras 5 reseñas
for i in range(5):
    print(dataset['train'][i])

<font color="darkorange" size="6"><b>10. Cómo entrenar un LLM para una tarea específica (Fine Tuning)</b></font>

In [ ]:
!pip install transformers datasets torch

In [ ]:
from datasets import load_dataset

# Cargar el conjunto de datos de cuentos en español
dataset = load_dataset("Fernandoefg/cuentos_es")

In [ ]:
# imprime la estructura
print(dataset)

# ver cuento 0
# print(dataset["train"][0]["content"])

# ver todos los titulos
titulos = [cuento["title"] for cuento in dataset["train"]]
print(titulos[:10])  # Muestra los primeros 10 títulos

# buscar por un titulo
# titulo_buscado = "El Principito"

# cuento = next((cuento for cuento in dataset["train"] if cuento["title"] == titulo_buscado), None)

# if cuento:
#     print(f"Título: {cuento['title']}\nAutor: {cuento['author']}\n\n{cuento['content']}")
# else:
#     print("No se encontró el cuento.")

In [ ]:
# Se tokeniza el texto y se añaden las etiquetas (labels)

from transformers import AutoTokenizer

# Cargar el tokenizador de un modelo GPT-2 en español
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")

# Tokenizar el texto de las historias y añadir labels
def tokenize_function(examples):
    tokens = tokenizer(examples["content"], truncation=True, padding="max_length", max_length=512)

    # Se necesita que labels sea igual a input_ids para la pérdida de entrenamiento
    tokens["labels"] = tokens["input_ids"].copy()

    return tokens

# Aplicar la tokenización al conjunto de datos
tokenized_datasets = dataset.map(tokenize_function, batched=True)



In [ ]:
from datasets import DatasetDict

# Como el conjunto no tiene explicito ttrain y test hacemos un split
dataset_split = tokenized_datasets["train"].train_test_split(test_size=0.2)

# Crear un nuevo DatasetDict con train y test
midataset = DatasetDict({
    "train": dataset_split["train"],
    "test": dataset_split["test"]
})

In [ ]:
from transformers import AutoModelForCausalLM

# Cargar el modelo GPT-2 en español para generación de texto
model = AutoModelForCausalLM.from_pretrained("datificate/gpt2-small-spanish")

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
output_dir="./resultados_historias",  # Carpeta donde se guardan los resultados
    run_name="entrenamiento_historias",  # Nombre único para el experimento en WandB
    report_to="none",  # Desactiva WandB si no lo necesitas
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=midataset["train"],
    eval_dataset=midataset["test"],
    tokenizer=tokenizer,
)

In [ ]:
# La tarea es tan demandante que no se puede llevar a cabo en colab

trainer.train()

In [ ]:
# Cargar el modelo ajustado
from transformers import pipeline

modelo_ajustado = pipeline("text-generation", model="./resultados_historias")

# Generar una historia a partir de una frase inicial
prompt = "Érase una vez en un bosque encantado,"
resultados = modelo_ajustado(prompt, max_length=200, num_return_sequences=1)

print(resultados[0]["generated_text"])

Como se puede ver las posibilidades son infinitas, aquí dejo una lista de tipos de entrenamiento para los que las librerías de huggingFace están diseñadas:

## Tareas de NLP para Modelos Preentrenados

1. **Clasificación de Texto**
   - **Descripción:** Asignar categorías predefinidas a fragmentos de texto.
   - **Ejemplos:** Clasificación de correos electrónicos como "spam" o "no spam", análisis de sentimientos en reseñas de productos.

2. **Reconocimiento de Entidades Nombradas (NER)**
   - **Descripción:** Identificar y clasificar entidades como nombres de personas, organizaciones o lugares dentro de un texto.
   - **Ejemplos:** En la frase "Apple lanzó el nuevo iPhone en California", reconocer "Apple" como organización y "California" como lugar.

3. **Análisis de Sentimientos**
   - **Descripción:** Determinar la polaridad emocional de un texto, como positiva, negativa o neutral.
   - **Ejemplos:** Analizar opiniones en redes sociales para evaluar la percepción pública de una marca.

4. **Traducción Automática**
   - **Descripción:** Convertir texto de un idioma a otro.
   - **Ejemplos:** Traducir documentos del inglés al español.

5. **Resumen de Texto**
   - **Descripción:** Generar una versión condensada de un texto más largo, manteniendo la información esencial.
   - **Ejemplos:** Resumir artículos de noticias o informes extensos.

6. **Respuesta a Preguntas (QA)**
   - **Descripción:** Proporcionar respuestas precisas a preguntas formuladas en lenguaje natural, basándose en un contexto dado.
   - **Ejemplos:** Responder "¿Quién es el presidente de Francia?" utilizando una base de datos de conocimiento.

7. **Análisis de Sintaxis y Dependencias**
   - **Descripción:** Descomponer oraciones para entender su estructura gramatical y las relaciones entre palabras.
   - **Ejemplos:** Identificar el sujeto, verbo y objeto en una oración.

8. **Detección de Lenguaje**
   - **Descripción:** Identificar el idioma en el que está escrito un texto.
   - **Ejemplos:** Determinar que un documento está en francés.

9. **Reconocimiento de Voz a Texto**
   - **Descripción:** Transcribir contenido hablado en texto escrito.
   - **Ejemplos:** Convertir grabaciones de entrevistas en texto para análisis.

10. **Conversión de Texto a Voz**
    - **Descripción:** Generar audio hablado a partir de texto escrito.
    - **Ejemplos:** Lectores de pantalla para personas con discapacidad visual.

11. **Detección de Parafraseo**
    - **Descripción:** Identificar si dos frases tienen el mismo significado.
    - **Ejemplos:** Detectar duplicados en respuestas de encuestas.

12. **Resolución de Coreferencias**
    - **Descripción:** Determinar qué palabras en un texto se refieren a la misma entidad.
    - **Ejemplos:** En "María dijo que ella vendría", identificar que "ella" se refiere a "María".

13. **Extracción de Información**
    - **Descripción:** Extraer datos estructurados específicos de textos no estructurados.
    - **Ejemplos:** Obtener fechas y lugares de eventos mencionados en artículos de noticias.

14. **Corrección Gramatical**
    - **Descripción:** Identificar y corregir errores gramaticales en un texto.
    - **Ejemplos:** Sugerir correcciones en redacciones estudiantiles.

15. **Desambiguación de Palabras**
    - **Descripción:** Determinar el significado correcto de una palabra que tiene múltiples sentidos, según el contexto.
    - **Ejemplos:** En "El banco está cerrado", decidir si "banco" se refiere a una entidad financiera o a un asiento.

16. **Generación de Código**
    - **Descripción:** Escribir código de programación basado en descripciones en lenguaje natural.
    - **Ejemplos:** Generar una función en Python que calcule la suma de una lista de números.

17. **Conversión de Código a Lenguaje Natural**
    - **Descripción:** Explicar en lenguaje natural lo que hace un fragmento de código.
    - **Ejemplos:** Describir la funcionalidad de una función en JavaScript.

18. **Detección de Humor o Sarcasmo**
    - **Descripción:** Identificar tonos humorísticos o sarcásticos en textos.
    - **Ejemplos:** Analizar tweets para detectar sarcasmo.

19. **Análisis de Temas**
    - **Descripción:** Identificar temas o tópicos predominantes en un conjunto de documentos.
    - **Ejemplos:** Descubrir que un conjunto de artículos de noticias trata principalmente sobre economía y salud.

20. **Generación de Preguntas**
    - **Descripción:** Crear preguntas relevantes basadas en un texto dado.
    - **Ejemplos:** Generar preguntas de comprensión lectora para un párrafo educativo.

21. **Conversión de Formato de Texto**
    - **Descripción:** Transformar texto entre diferentes formatos o estilos.
    - **Ejemplos:** Convertir texto plano en formato JSON o XML.

22. **Detección de Plagio**
    - **Descripción:** Identificar contenido copiado o similar a otros textos existentes.
    - **Ejemplos:** Verificar la originalidad de trabajos académicos.

23. **Análisis de Redes Sociales**
    - **Descripción:** Analizar contenido de plataformas sociales para obtener insights.
    - **Ejemplos:** Evaluar la reacción del público ante un evento reciente.

24. **Clasificación de Intenciones**
    - **Descripción:** Identificar la intención detrás de una consulta o mensaje.
    - **Ejemplos:** Determinar si una pregunta en un chatbot es una consulta de información o una solicitud de acción.

25. **Detección de Entidades de Producto**
    - **Descripción:** Identificar menciones de productos específicos en textos.
    - **Ejemplos:** Encontrar referencias a modelos de teléfonos en reseñas.

26. **Análisis de Competencia**
    - **Descripción:** Evaluar menciones y sentimientos hacia competidores en el mercado.
    - **Ejemplos:** Analizar cómo se perciben diferentes marcas en comentarios de clientes.

27. **Detección de Noticias Falsas**
    - **Descripción:** Identificar la veracidad de la información presentada en un texto.
    - **Ejemplos:** Evaluar si un artículo de noticias contiene información falsa o engañosa.

28. **Detección de Contenido Ofensivo**
    - **Descripción:** Identificar lenguaje inapropiado o dañino en textos.
    - **Ejemplos:** Filtrar comentarios ofensivos en plataformas en línea.

29. **Análisis de Discurso**
    - **Descripción:** Examinar textos para entender estructuras argumentativas o retóricas.
    - **Ejemplos:** Analizar discursos políticos para identificar argumentos clave.

30. **Generación de Descripciones de Imágenes**
    - **Descripción:** Crear descripciones textuales basadas en el contenido de imágenes.
    - **Ejemplos:** Describir el contenido de una fotografía
